In [2]:
import pandas as pd
import numpy as np
import pickle
import math

In [3]:
df=pd.read_parquet('valid_radek.parquet')

In [4]:
df.head()

,session,aid,ts,type
0,11098528,11830,1661119200,0
1,11098529,1105029,1661119200,0
2,11098530,264500,1661119200,0
3,11098530,264500,1661119288,0
4,11098530,409236,1661119369,0


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7683577 entries, 0 to 7683576
Data columns (total 4 columns):
 #   Column   Dtype
---  ------   -----
 0   session  int32
 1   aid      int32
 2   ts       int32
 3   type     uint8
dtypes: int32(3), uint8(1)
memory usage: 95.3 MB


In [6]:
len(df)

7683577

In [7]:
df.columns=[x.lower() for x in df.columns]

In [17]:
df=df.apply(pd.to_numeric)

In [18]:
df.to_parquet('valid_sample_full_cv_lastweek_cands_float.parquet')

In [8]:
df.to_csv('valid_radek.csv', index=False)

# Covisitation matrices from Chris Deotte

In [33]:
file_name = "top_20_buy2buy_cv.pkl"

top_20_buy2buy= pickle.load(open(file_name, "rb"))

In [34]:
result=[]
for key,items in top_20_buy2buy.items():
    for item in items:
        result.append((key,item))

KeyboardInterrupt: 

In [ ]:
result=pd.DataFrame(result, columns=['aid_click', 'aid_order'])

In [ ]:
len(result)

In [ ]:
result.to_csv('top_20_buy2buy_cv.csv', index=False)

In [2]:
import pandas as pd

In [3]:
df=pd.read_csv(r'submission_best.csv')
df2=pd.read_csv(r'submission_final.csv')

In [4]:
list(df[df['session_type'].apply(lambda x: x=='12905800_orders')]['labels'])

['749621 1321077 717616 1058210 881244 507572 568129 954732 746385 620804 448188 223067 926524 21429 297109 1563360 1617588 1113627 1731285 852172']

In [5]:
list(df2[df2['session_type'].apply(lambda x: x=='12905800_orders')]['labels'])

['749621 1321077 717616 1058210 507572 881244 568129 746385 620804 223067 448188 954732 21429 926524 1394967 821889 1731285 278961 852172 1563360']

In [6]:
df_new=df[df['session_type'].apply(lambda x: 'clicks' in x)]

In [7]:
df_new=pd.concat([df_new, df2[df2['session_type'].apply(lambda x: 'clicks' not in x)]])

In [8]:
df_new.head()

,session_type,labels
0,14564465_clicks,1411864 1768983 1264588 1585568 1466856 176444...
1,14564466_clicks,1207659 1853605 522206 1276297 15820 743992 16...
2,14564467_clicks,595052 1068921 1407136 781183 1851604 280328 1...
3,14564468_clicks,107579 507863 1430806 1640266 1502110 735394 1...
4,14564469_clicks,826556 1037628 77392 1453190 889659 242327 338...


In [9]:
df_new.to_csv('submission_79models_7best.csv', index=False)

In [12]:
df_new

,session_type,labels
0,14564465_clicks,1411864 1768983 1264588 1585568 1466856 176444...
1,14564466_clicks,1207659 1853605 522206 1276297 15820 743992 16...
2,14564467_clicks,595052 1068921 1407136 781183 1851604 280328 1...
3,14564468_clicks,107579 507863 1430806 1640266 1502110 735394 1...
4,14564469_clicks,826556 1037628 77392 1453190 889659 242327 338...
...,...,...
5015404,12901459_orders,1197632 1796103 620545 226413 102345 1271034 1...
5015405,12901460_orders,1254228 642635 1853056 403683 1370026 56934 13...
5015406,12901461_orders,1039352 1048554 909871 333973 702714 824989 13...
5015407,12901462_orders,75686 1170894 807569 824946 1580498 155079 133...


# Scoring single session for manual check

In [ ]:
import pandas as pd
import pickle

In [ ]:
df=pd.read_csv(r'c:\Users\AZUBA\Downloads\valid_0_7_3.csv.gz')
print('File read.')
df.columns=[x.lower() for x in df.columns]

In [ ]:
df[(df['session_id']==11133806)]

In [ ]:
file_name = "otto_xgb_model_10_3weeks_no_nulls.pkl"

#load
xgb_model= pickle.load(open(file_name, "rb"))

In [ ]:
a=df[(df['session_id']==11133806)]

In [ ]:
a['null_flag']=(a['avg_order_hours_from_click_avg']<99999).apply(int)

In [ ]:
a['XGB_SCORE']=xgb_model.predict_proba(a[xgb_model.feature_names_in_])[:, 1]

In [ ]:
a.to_csv('scored_session_11133806.csv')

# Valid analysis

In [ ]:
import pandas as pd
import numpy as np
import random
import mofr
import matplotlib.pyplot as plt
import seaborn as sns
from xgboost import XGBClassifier, plot_tree
import math
import pickle


import xgboost as xgb

In [ ]:
file_name = "otto_xgb_model_10_sessions_with_purchase.pkl"

#load
xgb_model= pickle.load(open(file_name, "rb"))

In [ ]:
col_target='target'

In [ ]:
valid=pd.read_parquet('valid_sample_full_float.parquet') 

In [ ]:
valid['null_flag']=(valid['avg_order_hours_from_click_avg']<99999).apply(int)

In [ ]:
valid['null_flag'].value_counts(normalize=True)

In [ ]:
valid['XGB_SCORE']=xgb_model.predict_proba(valid[xgb_model.feature_names_in_])[:, 1]

In [ ]:
print('The Lift on the valid set is: '+ str(mofr.metrics.lift(valid[col_target], valid['XGB_SCORE'])))
print('The gini on the valid set is: '+ str(mofr.metrics.gini(valid[col_target].apply(int), valid['XGB_SCORE'])))
print('The accuracy on the valid set is: '+ str(mofr.metrics.accuracy_score(valid[col_target].apply(int), valid['XGB_SCORE'].apply(lambda x: int(x>0.5)))))
print('\n')

In [ ]:
top20_valid=valid.groupby('session_id').apply(lambda x : x.sort_values(by = ['XGB_SCORE'], ascending = [False]).head(20).reset_index(drop = True)).reset_index(drop = True)
precision=pd.DataFrame(top20_valid.groupby('session_id')[col_target].apply(np.mean)).rename(columns={ col_target:'PRECISION'}).reset_index()
mean_precision=np.mean(precision['PRECISION'])
print(f'The mean precision on valid set is: {mean_precision}')

In [ ]:
recall=pd.DataFrame(top20_valid.groupby('session_id')[col_target].apply(np.sum)).merge(valid.groupby('session_id')['number_carts_orders'].apply(np.mean), how='inner', on='session_id')
mean_recall=np.mean(recall[recall['number_carts_orders']>0]['target']/recall[recall['number_carts_orders']>0]['number_carts_orders'])
print(f'The mean recall on valid set is: {mean_recall}')

In [ ]:
top20_valid=valid.groupby('session_id').apply(lambda x : x.sort_values(by = ['in_session','times_in_top20','XGB_SCORE'], ascending = [False, False,False]).head(20).reset_index(drop = True)).reset_index(drop = True)
precision=pd.DataFrame(top20_valid.groupby('session_id')[col_target].apply(np.mean)).rename(columns={ col_target:'PRECISION'}).reset_index()
mean_precision=np.mean(precision['PRECISION'])
print(f'The mean precision on valid set is: {mean_precision}')

In [ ]:
recall=pd.DataFrame(top20_valid.groupby('session_id')[col_target].apply(np.sum)).merge(valid.groupby('session_id')['number_carts_orders'].apply(np.mean), how='inner', on='session_id')
mean_recall=np.mean(recall[recall['number_carts_orders']>0]['target']/recall[recall['number_carts_orders']>0]['number_carts_orders'])
print(f'The mean recall on valid set is: {mean_recall}')

In [ ]:
valid['num_times_ordered_overall']=valid['num_times_ordered_overall'].fillna(0)

In [ ]:
# results=[]
# for col in xgb_model.feature_names_in_:
# #     print(col)
#     results.append((col, np.abs(mofr.metrics.gini(valid[col_target], valid[col]))))
  
# pd.DataFrame(results, columns=['Predictor', 'GINI']).sort_values(by='GINI', ascending=False)[0:30]

In [ ]:
cols=[
'sum_prob_order',
 'in_session',
 'avg_prob_order',
 'sum_num_sessions_ordered',
 #'min_order_hours_from_click_avg',
 'avg_num_sessions_ordered',
 #'avg_order_hours_from_click_avg',
 'num_times_ordered_overall',
 'sess_num_carts',
 #'max_order_hours_from_click_avg',
 #'sess_dayofmonth',
 'times_in_top20',
]

In [ ]:
result=[]
for col in cols:
    print(col)
    top20_valid=valid.groupby('session_id').apply(lambda x : x.sort_values(by = [col,'XGB_SCORE'], ascending = [ False,False]).head(20).reset_index(drop = True)).reset_index(drop = True)
    precision=pd.DataFrame(top20_valid.groupby('session_id')[col_target].apply(np.mean)).rename(columns={ col_target:'PRECISION'}).reset_index()
    mean_precision=np.mean(precision['PRECISION'])
    print(f'The mean precision on valid set is: {mean_precision}')
    recall=pd.DataFrame(top20_valid.groupby('session_id')[col_target].apply(np.sum)).merge(valid.groupby('session_id')['number_carts_orders'].apply(np.mean), how='inner', on='session_id')
    mean_recall=np.mean(recall[recall['number_carts_orders']>0]['target']/recall[recall['number_carts_orders']>0]['number_carts_orders'])
    print(f'The mean recall on valid set is: {mean_recall}')
    result.append((col,mean_precision, mean_recall))
    print('\n')

pd.DataFrame(result, columns=['pred', 'precision', 'recall']).sort_values(by = ['recall'], ascending = [False])

In [ ]:
result=[]
for col in cols:
    for col2 in cols:
        print(col,col2)
        top20_valid=valid.groupby('session_id').apply(lambda x : x.sort_values(by = [col, col2,'XGB_SCORE'], ascending = [ False,False,False]).head(20).reset_index(drop = True)).reset_index(drop = True)
        precision=pd.DataFrame(top20_valid.groupby('session_id')[col_target].apply(np.mean)).rename(columns={ col_target:'PRECISION'}).reset_index()
        mean_precision=np.mean(precision['PRECISION'])
    #     print(f'The mean precision on valid set is: {mean_precision}')
        recall=pd.DataFrame(top20_valid.groupby('session_id')[col_target].apply(np.sum)).merge(valid.groupby('session_id')['number_carts_orders'].apply(np.mean), how='inner', on='session_id')
        mean_recall=np.mean(recall[recall['number_carts_orders']>0]['target']/recall[recall['number_carts_orders']>0]['number_carts_orders'])
    #     print(f'The mean recall on valid set is: {mean_recall}')
        result.append((col,col2,mean_precision, mean_recall))
    #     print('\n')

pd.DataFrame(result, columns=['pred1', 'pred2', 'precision', 'recall']).sort_values(by = ['recall'], ascending = [False])

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_valid, y_train, y_valid= train_test_split(train, train[col_target], test_size=0.2, random_state=seed)

# Simple model experiment

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_valid, y_train, y_valid= train_test_split(valid, valid[col_target], test_size=0.5, random_state=1234)

In [ ]:
simple_model = XGBClassifier(c)
simple_model.fit(X_train.loc[:, cols], y_train, verbose=0, eval_metric='auc')

In [ ]:
valid['SIMPLE_SCORE']=simple_model.predict_proba(valid[simple_model.feature_names_in_])[:, 1]

In [ ]:
print('The Lift on the valid set is: '+ str(mofr.metrics.lift(X_valid[col_target], X_valid['SIMPLE_SCORE'])))
print('The gini on the valid set is: '+ str(mofr.metrics.gini(X_valid[col_target].apply(int), X_valid['SIMPLE_SCORE'])))
print('The accuracy on the valid set is: '+ str(mofr.metrics.accuracy_score(X_valid[col_target].apply(int), X_valid['SIMPLE_SCORE'].apply(lambda x: int(x>0.5)))))
print('\n')

In [ ]:
from xgboost import plot_tree
plot_tree(simple_model)
plt.show()

In [ ]:
top20_valid=X_train.groupby('session_id').apply(lambda x : x.sort_values(by = ['SIMPLE_SCORE'], ascending = [ False]).head(20).reset_index(drop = True)).reset_index(drop = True)
precision=pd.DataFrame(top20_valid.groupby('session_id')[col_target].apply(np.mean)).rename(columns={ col_target:'PRECISION'}).reset_index()
mean_precision=np.mean(precision['PRECISION'])
print(f'The mean precision on valid set is: {mean_precision}')

In [ ]:
print('The Lift on the valid set is: '+ str(mofr.metrics.lift(valid[col_target], valid['SIMPLE_SCORE'])))
print('The gini on the valid set is: '+ str(mofr.metrics.gini(valid[col_target].apply(int), valid['SIMPLE_SCORE'])))
print('The accuracy on the valid set is: '+ str(mofr.metrics.accuracy_score(valid[col_target].apply(int), valid['SIMPLE_SCORE'].apply(lambda x: int(x>0.5)))))
print('\n')

In [ ]:
recall=pd.DataFrame(top20_valid.groupby('session_id')[col_target].apply(np.sum)).merge(valid.groupby('session_id')['number_carts_orders'].apply(np.mean), how='inner', on='session_id')
mean_recall=np.mean(recall[recall['number_carts_orders']>0]['target']/recall[recall['number_carts_orders']>0]['number_carts_orders'])
print(f'The mean recall on valid set is: {mean_recall}')

# Evaluating metrics on valid sample

In [ ]:
import pandas as pd
import numpy as np
import random
import mofr
import matplotlib.pyplot as plt
import seaborn as sns
from xgboost import XGBClassifier, plot_tree
import math
import pickle


import xgboost as xgb

In [ ]:
file_name = "otto_xgb_model_10_4weeks_no_nulls.pkl"
#load
xgb_model= pickle.load(open(file_name, "rb"))

In [ ]:
col_target='target'

In [ ]:
valid=pd.read_parquet('valid_sample_full_float.parquet') 

In [ ]:
valid=pd.read_parquet('valid_sample_full_float.parquet')

In [ ]:
valid['XGB_SCORE']=xgb_model.predict_proba(valid[xgb_model.feature_names_in_])[:, 1]

In [ ]:
valid['null_flag']=(valid['avg_order_hours_from_click_avg']<99999).apply(int)

In [ ]:
a=valid.groupby('session_id').apply(lambda x : x.sort_values(by = ['in_session','times_in_top20','XGB_SCORE'], ascending = [False, False,False]).head(20).reset_index(drop = True)).reset_index(drop = True)
a['session_id']=a['session_id'].apply(lambda x: str(int(x)))
a['aid']=a['aid'].apply(lambda x: str(int(x)))
    
pred_df=pd.DataFrame(a.groupby('session_id')['aid'].apply(list).apply(' '.join)).reset_index(drop=False).rename(columns={'aid':'labels'})

In [ ]:
submission_clicks=pred_df.copy()
submission_carts=pred_df.copy()
submission_orders=pred_df.copy()

submission_clicks['session_type']=submission_clicks['session_id'].apply(lambda x: x+'_clicks')
submission_carts['session_type']=submission_carts['session_id'].apply(lambda x: x+'_carts')
submission_orders['session_type']=submission_orders['session_id'].apply(lambda x: x+'_orders')      

In [ ]:
submission_final=pd.concat([submission_clicks[['session_type', 'labels']],submission_carts[['session_type', 'labels']],submission_orders[['session_type', 'labels']]])

In [ ]:
submission_final

In [ ]:
# COMPUTE METRIC
score = 0
weights = {'clicks': 0.10, 'carts': 0.30, 'orders': 0.60}
for t in ['clicks','carts','orders']:
    sub = submission_final.loc[submission_final.session_type.str.contains(t)].copy()
    sub['session'] = sub.session_type.apply(lambda x: int(x.split('_')[0]))
    sub.labels = sub.labels.apply(lambda x: [int(i) for i in x.split(' ')[:20]])
    test_labels = pd.read_parquet('test_cv_labels.parquet')
    test_labels = test_labels.loc[test_labels['type']==t]
    test_labels = test_labels.merge(sub, how='inner', on=['session'])
    test_labels['hits'] = test_labels.apply(lambda df: len(set(df.ground_truth).intersection(set(df.labels))), axis=1)
    test_labels['gt_count'] = test_labels.ground_truth.str.len().clip(0,20)
    recall = test_labels['hits'].sum() / test_labels['gt_count'].sum()
    score += weights[t]*recall
    print(f'{t} recall =',recall)
    
print('=============')
print('Overall Recall =',score)
print('=============')

In [ ]:
!pip install jsonlines

In [ ]:
import pandas as pd
import jsonlines

In [ ]:
test_list=[]
n=1

test_list=[]
#Iterate over the json files
with jsonlines.open('valid_radek2.jsonl') as reader:
    #Iterate over the each line on the reader via enumerate
    for idx, result in enumerate(reader):
        
        for event in result['events']:
            test_list.append((result['session'], event['aid'], event['ts'], event['type']))
        #print(f"result: {result['session']}")
        #print(isinstance(result, dict))
        if n%1000000==0:
            print(n)
        n+=1
    test=pd.DataFrame(test_list, columns=['session', 'aid', 'ts', 'type'])
    test.to_csv('valid_radek2.csv', index=False)
    test_list=[]  

# Ensembling

In [3]:
import pandas as pd
import numpy as np
import random
import mofr
import matplotlib.pyplot as plt
import seaborn as sns
from xgboost import XGBClassifier, plot_tree
import math
import pickle
import lightgbm as lgb
from sklearn import linear_model

import xgboost as xgb

In [2]:
file_name_clicks = "otto_xgb_model_13_clicks.pkl"
file_name_carts = "otto_xgb_model_13_cv_576.pkl"
file_name_orders = "otto_xgb_model_13_cv_576.pkl"

file_name_lgbm="otto_lgb_model_13_cv.pkl"
file_name_ridgereg="otto_ridgeregression_model_13_cv.pkl"


# load
xgb_model_clicks= pickle.load(open(file_name_clicks, "rb"))
xgb_model_carts= pickle.load(open(file_name_carts, "rb"))
xgb_model_orders= pickle.load(open(file_name_orders, "rb"))

lgb_model=pickle.load(open(file_name_lgbm, "rb"))
rr_model=pickle.load(open(file_name_ridgereg, "rb"))

In [6]:
valid=pd.read_parquet('valid_sample_full_cv_3targets_experimental.parquet')
valid.columns=[x.lower() for x in valid.columns]
valid['null_flag']=(valid['avg_order_hours_from_click_avg']<99999).apply(int)
valid['in_session_carted_not_ordered']=((valid['in_session_carted']>0)&(valid['in_session_ordered']==0)).apply(int)
valid['in_session_clicked_not_carted']=((valid['in_session_clicked']>0)&(valid['in_session_carted']==0)).apply(int)
valid['XGB_SCORE_CLICKS']=xgb_model_clicks.predict_proba(valid[xgb_model_clicks.feature_names_in_])[:, 1]
valid['XGB_SCORE_CARTS']=xgb_model_carts.predict_proba(valid[xgb_model_carts.feature_names_in_])[:, 1]
valid['XGB_SCORE_ORDERS']=xgb_model_orders.predict_proba(valid[xgb_model_orders.feature_names_in_])[:, 1]
valid['LGB_SCORE']=lgb_model.predict(valid[lgb_model.feature_name()])
valid['RR_SCORE']=rr_model.decision_function(valid[lgb_model.feature_name()].fillna(0))
valid['RR_SCORE']=np.exp(valid['RR_SCORE']) / np.sum(np.exp(valid['RR_SCORE']))

In [9]:
valid['XGB_RANK']=valid.groupby('session_id')['XGB_SCORE_ORDERS'].rank(method="dense", ascending=False)
valid['LGB_RANK']=valid.groupby('session_id')['LGB_SCORE'].rank(method="dense", ascending=False)
valid['RR_RANK']=valid.groupby('session_id')['RR_SCORE'].rank(method="dense", ascending=False)

In [11]:
valid[valid['session_id']==11098546.0]

,target,target_clicks,target_carts,target_orders,session_id,aid,min_top20,avg_top20,max_top20,in_session,...,in_session_carted_not_ordered,in_session_clicked_not_carted,XGB_SCORE_CLICKS,XGB_SCORE_CARTS,XGB_SCORE_ORDERS,LGB_SCORE,RR_SCORE,XGB_RANK,LGB_RANK,RR_RANK
0,0.0,0.0,0.0,0.0,11098546.0,605.0,9.0,50.000000,91.0,0.0,...,0,0,0.001949,0.000160,0.000160,0.152916,3.465870e-08,297.0,222.0,620.0
1,0.0,0.0,0.0,0.0,11098546.0,665.0,71.0,71.000000,71.0,0.0,...,0,0,0.000588,0.000047,0.000047,0.032125,3.462064e-08,945.0,829.0,1789.0
2,0.0,0.0,0.0,0.0,11098546.0,1089.0,16.0,45.666667,82.0,0.0,...,0,0,0.001342,0.000052,0.000052,0.099905,3.463923e-08,856.0,358.0,1218.0
3,0.0,0.0,0.0,0.0,11098546.0,1303.0,35.0,60.500000,87.0,0.0,...,0,0,0.000919,0.000076,0.000076,0.060689,3.465171e-08,596.0,534.0,832.0
4,0.0,0.0,0.0,0.0,11098546.0,1398.0,15.0,15.000000,15.0,0.0,...,0,0,0.000740,0.000677,0.000677,0.183634,3.466962e-08,58.0,174.0,366.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4041,0.0,0.0,0.0,0.0,11098546.0,1849590.0,26.0,51.222222,72.0,0.0,...,0,0,0.003546,0.000225,0.000225,0.125579,3.465085e-08,195.0,280.0,856.0
4042,0.0,0.0,0.0,0.0,11098546.0,1849958.0,0.0,33.027778,96.0,1.0,...,0,1,0.038476,0.000182,0.000182,0.553500,3.575886e-08,256.0,17.0,34.0
4043,0.0,0.0,0.0,0.0,11098546.0,1851023.0,92.0,92.000000,92.0,0.0,...,0,0,0.000300,0.000022,0.000022,0.021621,3.462334e-08,1742.0,1041.0,1701.0
4044,0.0,0.0,0.0,0.0,11098546.0,1853067.0,80.0,80.000000,80.0,0.0,...,0,0,0.000257,0.000018,0.000018,0.021666,3.460117e-08,1872.0,1038.0,2197.0


In [12]:
valid['SUM_RANKS']=valid['XGB_RANK']+valid['LGB_RANK']+valid['RR_RANK']

In [18]:
a=valid.groupby('session_id').apply(lambda x : x.sort_values(by = ['SUM_RANKS'], ascending = [True]).head(20).reset_index(drop = True)).reset_index(drop = True)
a['session_id']=a['session_id'].apply(lambda x: str(int(x)))
a['aid']=a['aid'].apply(lambda x: str(int(x)))
    
pred_df=pd.DataFrame(a.groupby('session_id')['aid'].apply(list).apply(' '.join)).reset_index(drop=False).rename(columns={'aid':'labels'})

In [19]:
submission_clicks=pred_df.copy()
submission_carts=pred_df.copy()
submission_orders=pred_df.copy()

submission_clicks['session_type']=submission_clicks['session_id'].apply(lambda x: x+'_clicks')
submission_carts['session_type']=submission_carts['session_id'].apply(lambda x: x+'_carts')
submission_orders['session_type']=submission_orders['session_id'].apply(lambda x: x+'_orders')    

In [20]:
submission_final=pd.concat([submission_clicks[['session_type', 'labels']],submission_carts[['session_type', 'labels']],submission_orders[['session_type', 'labels']]])

In [21]:
# COMPUTE METRIC
score = 0
weights = {'clicks': 0.10, 'carts': 0.30, 'orders': 0.60}
for t in ['clicks','carts','orders']:
    sub = submission_final.loc[submission_final.session_type.str.contains(t)].copy()
    sub['session'] = sub.session_type.apply(lambda x: int(x.split('_')[0]))
    sub.labels = sub.labels.apply(lambda x: [int(i) for i in x.split(' ')[:20]])
    test_labels = pd.read_parquet('test_cv_labels.parquet')
    test_labels = test_labels.loc[test_labels['type']==t]
    test_labels = test_labels.merge(sub, how='inner', on=['session'])
    test_labels['hits'] = test_labels.apply(lambda df: len(set(df.ground_truth).intersection(set(df.labels))), axis=1)
    test_labels['gt_count'] = test_labels.ground_truth.str.len().clip(0,20)
    recall = test_labels['hits'].sum() / test_labels['gt_count'].sum()
    score += weights[t]*recall
    print(f'{t} recall =',recall)
    
print('=============')
print('Overall Recall =',score)
print('=============')

clicks recall = 0.49764131826803054
carts recall = 0.43548186631534463
orders recall = 0.6604910714285714
Overall Recall = 0.5767033345785493
